[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/experimental/algos-and-libraries/bertopic/01_topic_modeling.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/experimental/algos-and-libraries/bertopic/01_topic_modeling.ipynb)

In [ ]:
!pip install bertopic

We will use the r/python data downloaded before. If loading from file:

```python
import pandas as pd

data = pd.read_csv('python.csv', sep='|')
```

Otherwise, load directly from HuggingFace Hub like so:

In [1]:
from datasets import load_dataset

data = load_dataset('jamescalam/reddit-python', split='train')
data

Using custom data configuration jamescalam--reddit-python-107f1094f98d4fba
Reusing dataset json (/Users/jamesbriggs/.cache/huggingface/datasets/json/jamescalam--reddit-python-107f1094f98d4fba/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


Dataset({
    features: ['sub', 'title', 'selftext', 'upvote_ratio', 'id', 'created_utc'],
    num_rows: 933
})

Let's remove rows where `selftext` is *NaN* or very short. If using dataframe use:

```python
data = data[data['selftext'].str.len() > 30].reset_index()
```

Else...

In [2]:
data = data.filter(lambda x: True if len(x['selftext']) > 30 else 0)
data

Loading cached processed dataset at /Users/jamesbriggs/.cache/huggingface/datasets/json/jamescalam--reddit-python-107f1094f98d4fba/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-093f5a73cbc33e37.arrow


Dataset({
    features: ['sub', 'title', 'selftext', 'upvote_ratio', 'id', 'created_utc'],
    num_rows: 622
})

In [3]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# we add this to remove stopwords, for lower volumes of data stopwords can cause issues
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

# deal with df if needed
if type(data['selftext']) is list:
    text = data['selftext']
else:
    text = data['selftext'].tolist()

model = BERTopic(
    vectorizer_model=vectorizer_model,
    language='english', calculate_probabilities=True,
    verbose=True
)
topics, probs = model.fit_transform(text)

Batches:   5%|▌         | 1/20 [00:07<02:17,  7.21s/it]


This returns two lists, `topics` contains a one-to-one mapping of our titles to topic numbers, and `probs` contains a list of probabilities of a thread belonging to each topic (based on its `selftext`).

In [ ]:
for i in range(5):
    print(f"{topics[i]}: {data['selftext'][i]}")

We can see the top words found in each topic using `get_topic_info`.

In [ ]:
freq = model.get_topic_info()
freq.head(10)

The top `-1` topic is generally assumed to be irrelevant, if we hadn't used the `vectorizer_model` this would typically be full of stopwords. In our case this seems to be the *"most generic"* of topics, about Python, code, and data.

We can also visualize how related topics are using `visualize_topics`.

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_hierarchy()

In [ ]:
model.visualize_barchart()

---